<a href="https://colab.research.google.com/github/jfcaro/Vacunacion_COVID_19_ESP/blob/main/Ritmo_vacunacion_Covid19v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Ritmo de vacunación España (02/05/2021)
 Cálculo del % de población por comunidad vacunada (pauta completa)<br>
 Cálculo de fecha para conseguir 100% de vacuación manteniendo velocidad actual <br>
 Cálculo de la velocidad diaria para conseguir vacunación del 70% de población <br>
 v2: Actualizado para nueva versión fichero de datos vacunaciones COVID.
 <br>
 
 @jfrcaro

In [1]:
#Instalar liberías

# odfpy necesario para tratar hojas de cálculo en formato odf
!pip install odfpy
# !pip install pandas
# !pip install datetime

# importar librerías 
import pandas as pd
from datetime import datetime, timedelta

### ¿Cuál es la población por comunidad?
Para la obtención de la población por comunidad se emplea una tabla de la wikipedia : https://es.wikipedia.org/wiki/Comunidad_aut%C3%B3noma

In [2]:
# Obtencion carga de las tablas de la página https://es.wikipedia.org/wiki/Comunidad_aut%C3%B3noma'
comunidades = pd.read_html('https://es.wikipedia.org/wiki/Comunidad_aut%C3%B3noma')

In [3]:
# Obtención de tabla con población por comunidad sin totales, incluyendo ajuste de valores

# Seleccionamos la tabla 5 de wikipedia que es la relacionada con población
comunidad = comunidades[4]

# Ajustes diversos en el campo Población(hab.)
comunidad["Población(hab.)"] = comunidad["Población(hab.)"].astype(str)
comunidad["Población(hab.)"]= comunidad["Población(hab.)"].apply(lambda x: str(x).replace(' ', ''))
comunidad["Población(hab.)"]= comunidad["Población(hab.)"].apply(lambda x: str(x).replace(u'\xa0', u''))

# Mostrar tabla 
comunidad

,Posición,Nombre,Población(hab.),%,Densidad(hab./km²)
0,1,Andalucía,8379248,1799,9602
1,2,Cataluña,7596131,1622,23533
2,3,Comunidad de Madrid,6576009,1397,81117
3,4,Comunidad Valenciana,4959243,1061,21249
4,5,Galicia,2700970,582,9158
5,6,Castilla y León,2407650,521,2574
6,7,País Vasco,2198657,471,30331
7,8,Canarias,2126779,451,28308
8,9,Castilla-La Mancha,2025510,453,2662
9,10,Región de Murcia,1477946,316,12996


### ¿Cuáles son los últimos datos sobre vacunación? 
Para la obtención de las estadísticas de vacunación se emplea tabla de https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm

In [4]:
# Obtención de tabla de vacunación. Viene en formato ods por lo que se requiere importar odfpy

# Obtención del día actual
hoy = datetime.today()

# Día de la semana de hoy
diasemana = hoy.weekday()

# Se coge el día anterior para asegurar que existe el fichero, en el caso de fin de semana y lunes se toma viernes
if diasemana in [1,2,3,4,5]:
    hoy = datetime.today()-timedelta(days = 1)
elif diasemana == 0: 
    hoy = datetime.today()-timedelta(days = 3)
elif diasemana == 6:
    hoy = datetime.today()-timedelta(days = 2)

# Función para convertir mes y día a dos cifras
def conversion(numero):
    if numero >= 10: 
        return str(numero)
    else:
        return "0"+str(numero)
    
# Nombre del fichero a descargar desde www.mscbs.gob.es 
nombre_fichero = "Informe_Comunicacion_"+str(hoy.year)+ conversion(hoy.month)+ conversion(hoy.day)+".ods"

# descargar el fichero de hoy en formato odf para lo que hay que instalar odfpy
try:     
    vacunacion = pd.read_excel("https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/"+nombre_fichero, engine="odf")
except:
    print ("No se puede cargar el fichero: Verificar que existe {}".format("https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/documentos/"+ nombre_fichero))

In [5]:
# Cambio de nombre de columna Unnamed  por Comunidad
vacunacion.rename(columns={'Unnamed: 0':'Comunidad'}, inplace=True)

### ¿Cómo fusionar las tablas?
Para fusionar las tablas es necesario encontrar un campo común en este caso en nombre de la comunidad <br>
A continuación se realizan algunas modificaciones para poder fusionar

In [6]:
# Cambiar del dataframe vacunación el nombre de algunas comunidades para igualar al de comunidad y poder fusionar
vacunacion.loc[15, 'Comunidad'] = "Comunidad Foral de Navarra"
vacunacion.loc[9, 'Comunidad'] = "Comunidad Valenciana"
vacunacion.loc[13, 'Comunidad'] = "Comunidad de Madrid"
vacunacion.loc[6, 'Comunidad'] = "Castilla y León"
vacunacion.loc[7, 'Comunidad'] = "Castilla-La Mancha"
vacunacion.loc[3, 'Comunidad'] = "Islas Baleares"
vacunacion.loc[2, 'Comunidad'] = "Principado de Asturias"
vacunacion.loc[14, 'Comunidad'] = "Región de Murcia"
vacunacion.loc[20, 'Comunidad'] = "TOTAL"

In [7]:
# % sobre entregadas en tanto por 100
vacunacion['% sobre entregadas']=vacunacion['% sobre entregadas']*100

# añadir fecha media en la fila de totales
vacunacion.loc[20, "Fecha de la última vacuna registrada (2)"]= vacunacion["Fecha de la última vacuna registrada (2)"].mean()

In [8]:
# Mostar la table de vacunación
vacunacion

,Comunidad,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Dosis entregadas Janssen (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas con al menos 1 dosis,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2)
0,Andalucía,1907190,248500,669700,48650,2874040,2831912,98.534189,2004070,846489,2021-04-29
1,Aragón,393435,51800,105200,7600,558035,524214,93.939269,377520,150773,2021-04-29
2,Principado de Asturias,372965,47600,80600,5850,507015,474656,93.617743,324880,152091,2021-04-29
3,Islas Baleares,214470,27900,92500,6700,341570,319347,93.493867,236608,86442,2021-04-29
4,Canarias,420000,54700,172100,12500,659300,608346,92.271500,448628,165908,2021-04-29
5,Cantabria,176235,22500,46000,3300,248035,224037,90.324753,156532,67988,2021-04-29
6,Castilla y León,877215,113700,189500,13800,1194215,1067062,89.352587,744217,328108,2021-04-29
7,Castilla-La Mancha,552105,71900,161800,11700,797505,725719,90.998677,513927,216193,2021-04-29
8,Cataluña,1887830,251300,615500,44700,2799330,2590635,92.544823,1820712,783350,2021-04-29
9,Comunidad Valenciana,1136260,151600,400000,29000,1716860,1666527,97.068311,1197668,478277,2021-04-29


In [9]:
# Fusión tabla vacunacion y comunidad a través del nombre de la comunidad 
completa = vacunacion.merge(comunidad, left_on='Comunidad', right_on='Nombre')

### Cálculo de indicadores
%vacunado : Es el porcentaje de vacunados (pauta completa) <br><br>
fecha_restante: Es la fecha para completar el 100% al ritmo actual que son los vacunados actuales en el tiempo desde inicio de vacunación (27/12-2020)<br>
+ info: https://elpais.com/sociedad/2020-12-18/la-vacunacion-en-espana-arrancara-el-27-de-diciembre.html <br>

ritmo (vacunaciones completas por día) 70%: es el ritmo en vacunaciones pauta completa por día a aplicar desde el día posterior al cálculo para llegar <br>
al objetivo del 70% de la población al final del verano (21/09/2021)
+ info: https://www.eldiario.es/politica/pedro-sanchez-defiende-proceso-vacunacion-confia-llegara-70-poblacion-verano_1_7036447.html
+ info: https://m.europapress.es/eseuropa/noticia-comision-europea-von-der-leyen-dice-fecha-limite-vacunar-70-poblacion-adulta-21-septiembre-20210212153122.html


In [10]:
# Se añade columna para % vacunación por comunidad 
completa['Nº Personas vacunadas(pauta completada)']= completa['Nº Personas vacunadas(pauta completada)'].astype(int)
completa['Población(hab.)']= completa['Población(hab.)'].astype(int)
completa['%vacunado'] = (completa['Nº Personas vacunadas(pauta completada)']/completa['Población(hab.)'])*100

In [11]:
# Se añade columna tiempo estimado restante en función de velocidad desde inicio de vacunación (27-12-2020)
fecha_inicio_vacunacion = datetime(2020,12,27)
completa['dias_desde_inicio'] =  completa['Fecha de la última vacuna registrada (2)'].apply(lambda x: ((x - fecha_inicio_vacunacion).days))
completa['dias_restantes'] =completa['dias_desde_inicio']/(completa['%vacunado']/100)
completa['fecha_restante'] =completa['dias_restantes'].apply(lambda x:(hoy + timedelta(days = x)).strftime('%d/%m/%Y'))                                                         

In [12]:
# Se añade velocidad en personas vacunadas(pauta completa) al día para llegar al 70% el 21 de Septiembre de 2021
verano = datetime(2021,9,21)
dias_hasta_verano = (verano - datetime.today()).days
completa['ritmo (vacunaciones completas por día) 70%']=(completa['Población(hab.)']*0.7-completa['Nº Personas vacunadas(pauta completada)'])/dias_hasta_verano

In [13]:
# Columnas a incluir en la visualización final 
columnas_vacunacion = list(vacunacion.columns)
columnas_poblacion = ['Población(hab.)']
columnas_nuevas = ['%vacunado', 'fecha_restante', 'ritmo (vacunaciones completas por día) 70%']
total_columnas = columnas_vacunacion + columnas_poblacion + columnas_nuevas

In [14]:
# Visualizar Dataframe
completa[total_columnas]

,Comunidad,Dosis entregadas Pfizer (1),Dosis entregadas Moderna (1),Dosis entregadas AstraZeneca (1),Dosis entregadas Janssen (1),Total Dosis entregadas (1),Dosis administradas (2),% sobre entregadas,Nº Personas con al menos 1 dosis,Nº Personas vacunadas(pauta completada),Fecha de la última vacuna registrada (2),Población(hab.),%vacunado,fecha_restante,ritmo (vacunaciones completas por día) 70%
0,Andalucía,1907190,248500,669700,48650,2874040,2831912,98.534189,2004070,846489,2021-04-29,8379248,10.102207,30/08/2024,35595.635461
1,Aragón,393435,51800,105200,7600,558035,524214,93.939269,377520,150773,2021-04-29,1307984,11.527129,01/04/2024,5424.225532
2,Principado de Asturias,372965,47600,80600,5850,507015,474656,93.617743,324880,152091,2021-04-29,1028135,14.792902,10/08/2023,4025.556738
3,Islas Baleares,214470,27900,92500,6700,341570,319347,93.493867,236608,86442,2021-04-29,1128139,7.662354,22/09/2025,4987.626241
4,Canarias,420000,54700,172100,12500,659300,608346,92.271500,448628,165908,2021-04-29,2126779,7.800905,24/08/2025,9381.824823
5,Cantabria,176235,22500,46000,3300,248035,224037,90.324753,156532,67988,2021-04-29,580067,11.720715,15/03/2024,2397.580851
6,Castilla y León,877215,113700,189500,13800,1194215,1067062,89.352587,744217,328108,2021-04-29,2407650,13.627728,20/10/2023,9625.865248
7,Castilla-La Mancha,552105,71900,161800,11700,797505,725719,90.998677,513927,216193,2021-04-29,2025510,10.673509,26/06/2024,8522.439716
8,Cataluña,1887830,251300,615500,44700,2799330,2590635,92.544823,1820712,783350,2021-04-29,7596131,10.312487,05/08/2024,32155.614894
9,Comunidad Valenciana,1136260,151600,400000,29000,1716860,1666527,97.068311,1197668,478277,2021-04-29,4959243,9.644153,27/10/2024,21228.319858


### Se muestran en pantalla los indicadores calculados

In [15]:
# Indicadores para España
print ("A día de {} se ha vacunado (pauta completa) al {:,.2f}% de la población española.".format(str(hoy.strftime('%d/%m/%Y')), float(completa["%vacunado"][completa['Comunidad']=='TOTAL'].values[0])))
print ("El ritmo de vacunaciones en España desde el {} ha de ser de {:,.2f} vacunaciones-pauta completa/día para vacunar al 70% de la población a 21/09/2021.".format(str(hoy.strftime('%d/%m/%Y')), completa["ritmo (vacunaciones completas por día) 70%"][completa['Comunidad']=='TOTAL'].values[0]))

personas_pautacompleta = float(completa["Nº Personas vacunadas(pauta completada)"][completa['Comunidad']=='TOTAL'].values[0])
dias_desde_inicio = completa['dias_desde_inicio'][completa['Comunidad']=='TOTAL'].values[0] 
personas_por_dia= personas_pautacompleta/dias_desde_inicio

print ("Desde el inicio de la vacunación van {} días y se han vacunado (pauta completa) a {:,.0f} personas, lo que hace una media de {:,.2f} vacuanados por día.".format(dias_desde_inicio, personas_pautacompleta, personas_por_dia))
print ("Al ritmo anterior de vacunados por día se cubrirá el 100% de la población el {}.".format(completa["fecha_restante"][completa['Comunidad']=='TOTAL'].values[0]))

A día de 30/04/2021 se ha vacunado (pauta completa) al 10.04% de la población española.
El ritmo de vacunaciones en España desde el 30/04/2021 ha de ser de 198,576.12 vacunaciones-pauta completa/día para vacunar al 70% de la población a 21/09/2021.
Desde el inicio de la vacunación van 123 días y se han vacunado (pauta completa) a 4,689,766 personas, lo que hace una media de 38,128.18 vacuanados por día.
Al ritmo anterior de vacunados por día se cubrirá el 100% de la población el 06/09/2024.


In [16]:
# Indicadores por comunidad 

# Se obtienen las comunidades en lista y se le quita la fila de totales
comunidades = sorted(completa['Comunidad'].to_list()[0:-1])

In [17]:
# Se recorren todas las comunidades
for comunidad in comunidades: 
    print ("** DATOS DE LA COMUNIDAD AUTÓNOMA: {}".format(comunidad.upper()))
    print ("A día de {} se ha vacunado (pauta completa) al {:,.2f}% de la población de la comunidad.".format(str(hoy.strftime('%d/%m/%Y')), float(completa["%vacunado"][completa['Comunidad']==comunidad].values[0])))
    print ("El ritmo de vacunaciones la comunidad desde el {} ha de ser de {:,.2f} vacunaciones-pauta completa/día para vacunar al 70% de su población a 21/09/2021.".format(str(hoy.strftime('%d/%m/%Y')), completa["ritmo (vacunaciones completas por día) 70%"][completa['Comunidad']==comunidad].values[0]))

    personas_pautacompleta = float(completa["Nº Personas vacunadas(pauta completada)"][completa['Comunidad']==comunidad].values[0])
    dias_desde_inicio = completa['dias_desde_inicio'][completa['Comunidad']==comunidad].values[0] 
    personas_por_dia= personas_pautacompleta/dias_desde_inicio

    print ("Desde el inicio de la vacunación van {} días y se han vacunado (pauta completa) a {:,.0f} personas, lo que hace una media de {:,.2f} vacuanados por día.".format(dias_desde_inicio, personas_pautacompleta, personas_por_dia))
    print ("Al ritmo anterior de vacunados por día se cubrirá el 100% de su población el {}.".format(completa["fecha_restante"][completa['Comunidad']==comunidad].values[0]))
    print ("\n")
   

** DATOS DE LA COMUNIDAD AUTÓNOMA: ANDALUCÍA
A día de 30/04/2021 se ha vacunado (pauta completa) al 10.10% de la población de la comunidad.
El ritmo de vacunaciones la comunidad desde el 30/04/2021 ha de ser de 35,595.64 vacunaciones-pauta completa/día para vacunar al 70% de su población a 21/09/2021.
Desde el inicio de la vacunación van 123 días y se han vacunado (pauta completa) a 846,489 personas, lo que hace una media de 6,882.02 vacuanados por día.
Al ritmo anterior de vacunados por día se cubrirá el 100% de su población el 30/08/2024.


** DATOS DE LA COMUNIDAD AUTÓNOMA: ARAGÓN
A día de 30/04/2021 se ha vacunado (pauta completa) al 11.53% de la población de la comunidad.
El ritmo de vacunaciones la comunidad desde el 30/04/2021 ha de ser de 5,424.23 vacunaciones-pauta completa/día para vacunar al 70% de su población a 21/09/2021.
Desde el inicio de la vacunación van 123 días y se han vacunado (pauta completa) a 150,773 personas, lo que hace una media de 1,225.80 vacuanados por dí

### ¿Quieres la tabla en excel?

In [18]:
# Exporta dataframe completo a excel
completa[total_columnas].to_excel('./Vacunacion.xlsx', 'Datos',index = False)

### Para continuar
Afinar el cálculo del 100% de la población se podría calcular la última velocidad conseguida como vacunados día n - vacunados día (n-1) y aplicar ese ritmo al <br>
cálculo de fecha restante<br><br>
Obtener todas tablas generadas en https://www.mscbs.gob.es/profesionales/saludPublica/ccayes/alertasActual/nCov/vacunaCovid19.htm y ver la evolución temporal <br> 
de los indicadores<br><br>
Identificar concepto de "Población adulta" y obtención de la población adulta por comunidad para recalcular el objetivo al 70% de la población adulta

